# cui 2019

## 注意

1. `rpy2`来调用R代码，需要设`R_HOME`，否则会崩溃。
2  `%load_ext rpy2.ipython`之后，可以有`%%R`开头的纯R单元格，可以用`-i`传数据给R，或者`-o`提取数据，详情问ChatGPT。

In [2]:
# 尝试使用同意的合并数据系统
import os
import re
import pandas as pd
import numpy as np
import seaborn as sns
import sys

sys.path.insert(0, '..')  # 将上一层目录添加到系统路径

from src.reader import read_cfps, find_files, find_cfps

data_path = os.getenv('data_path')


## 构造孩子-母亲对应表


1. 从家庭信息库中，选择女性，以及她的子女
2. 宽表转长表，一位母亲对应多名子女
3. 

In [7]:
# 同一个变量，在不同年份数据中的变量名
var_list = pd.read_excel('./var_list.xlsx');var_list

,var,note,db,short_name,2020,2018,2016,2014,2012,2010
0,pid,个人代码,NaN,pid,pid,pid,pid,pid,pid,pid
1,wav,wav,NaN,wav,wav,wav,wav,wav,wav,wav
2,School enrollment,儿童入学,NaN,enroll,wc1,wc1_b_2,pc1_b_1,wf3m,NaN,NaN
3,School enrollment,青年入学,NaN,enroll,qc1,qc1,pc1,wc01,NaN,NaN
4,Cognition-math,数学成绩,NaN,math,wf502,wf502,wf502,wf502,NaN,NaN
5,Mother's education,最高学历,NaN,edu,w01,w01,pw1r,te4,NaN,NaN
6,Mother's age,年龄,NaN,age,age,age,age,cfps2014_age,NaN,NaN
7,Mother's year of birth,出生年份,NaN,birth_y,ibirthy,ibirthy,cfps_birthy,cfps_birthy,NaN,NaN
8,Mother's ethnicity (Han=1),民族,NaN,eth,qa701code,qa701code,pa701code,qa701code,NaN,NaN
9,Child's age,年龄,NaN,age,age,age,cfps_age,cfps2014_age,NaN,NaN


In [8]:
all_family_df_list = find_files(data_path,r'.*famconf.*\.dta$');all_family_df_list

['C:\\Users\\lee\\nutcloud\\Data\\cfps\\2012\\cfps2012famconf_092015.dta',
 'C:\\Users\\lee\\nutcloud\\Data\\cfps\\2014\\cfps2014famconf_170630.dta',
 'C:\\Users\\lee\\nutcloud\\Data\\cfps\\2016\\cfps2016famconf_201804.dta',
 'C:\\Users\\lee\\nutcloud\\Data\\cfps\\2018\\cfps2018famconf_202008.dta',
 'C:\\Users\\lee\\nutcloud\\Data\\cfps\\2020\\cfps2020famconf_202301.dta']

In [9]:
path = all_family_df_list[-1]
df = read_cfps(path);df.head()

C:\Users\lee\nutcloud\Data\cfps\2020\cfps2020famconf_202301.dta
feather存在，读取feather。


,fid20,fid18,fid16,fid14,fid12,fid10,fid_base,subpopulation,subsample,psu,...,cyear20,cmonth20,iwmode20,kz1pid20,sresppid20,interviewerid20,genetype20,familysize20,releaseversion,wav
0,100051.0,100051.0,100051.0,100051.0,-8.0,-8.0,110043.0,6.0,1.0,45.0,...,2020.0,7.0,2.0,100051502.0,100051502.0,427784.0,0.0,3.0,1.0,2020
1,100051.0,100051.0,100051.0,100051.0,-8.0,-8.0,110043.0,6.0,1.0,45.0,...,2020.0,7.0,2.0,100051502.0,100051502.0,427784.0,0.0,3.0,1.0,2020
2,100051.0,100051.0,100051.0,100051.0,110043.0,110043.0,110043.0,6.0,1.0,45.0,...,2020.0,7.0,2.0,100051502.0,100051502.0,427784.0,1.0,3.0,1.0,2020
3,100125.0,100125.0,100125.0,100125.0,110147.0,110147.0,110147.0,6.0,1.0,45.0,...,2020.0,7.0,2.0,110147102.0,110147102.0,823876.0,1.0,1.0,1.0,2020
4,100160.0,100160.0,-8.0,-8.0,-8.0,-8.0,120009.0,6.0,1.0,79.0,...,2020.0,8.0,2.0,100160601.0,100160601.0,136496.0,0.0,3.0,1.0,2020


In [89]:
year = 2018

list(var_list[year])


['pid',
 'wav',
 'wc1_b_2',
 'qc1',
 'wf502',
 'w01',
 'age',
 'ibirthy',
 'qa701code',
 'age',
 'gender',
 'tb2_a_p',
 'qa401a_code']

In [90]:
[c for c in df.columns if c in list(var_list[year])]

['pid', 'wav']

In [3]:
from src.utils import DataRenamer

dr = DataRenamer("./var_list.xlsx")
dr.get_var_list()

,var,note,db,short_name,2020,2018,2016,2014,2012,2010
0,pid,个人代码,NaN,pid,pid,pid,pid,pid,pid,pid
1,wav,wav,NaN,wav,wav,wav,wav,wav,wav,wav
2,School enrollment,儿童入学,NaN,enroll,wc1,wc1_b_2,pc1_b_1,wf3m,NaN,NaN
3,School enrollment,青年入学,NaN,enroll2,qc1,qc1,pc1,wc01,NaN,NaN
4,Cognition-math,数学成绩,NaN,math,wf502,wf502,wf502,wf502,NaN,NaN
5,Mother's education,完成教育年限,NaN,eduy,cfps2020eduy_im,cfps2018eduy_im,cfps2016eduy_im,cfps2014eduy_im,NaN,NaN
6,Mother's age,年龄,NaN,age,age,age,age,cfps2014_age,NaN,NaN
7,Mother's year of birth,出生年份,NaN,birth_y,ibirthy,ibirthy,cfps_birthy,birthy,NaN,NaN
8,Mother's ethnicity (Han=1),民族,NaN,eth,qa701code,qa701code,pa701code,qa701code,NaN,NaN
9,Child's age,年龄,NaN,age,age,age,cfps_age,cfps2014_age,NaN,NaN


In [217]:
def process_mother_children(df,code = 'pid_a_c'):

    wav = df['wav'][0]
    
    if wav <= 2016:
        code = 'pid_c'

    # 对于家庭数据，选择女性和她的全部子女数据
    children_cols = [s for s in df.columns if s.startswith(code)]
    df_female = df[['pid','wav','gender'] + children_cols].query(
        'gender == 0')

    df_female = df_female.applymap(lambda x: np.nan if (x < 0 or x == 79) else x)
    return df_female.drop('gender', axis=1)


In [218]:
all_family_df_list = find_files(data_path,r'.*famconf.*\.dta$');all_family_df_list


['C:\\Users\\lee\\nutcloud\\Data\\cfps\\2012\\cfps2012famconf_092015.dta',
 'C:\\Users\\lee\\nutcloud\\Data\\cfps\\2014\\cfps2014famconf_170630.dta',
 'C:\\Users\\lee\\nutcloud\\Data\\cfps\\2016\\cfps2016famconf_201804.dta',
 'C:\\Users\\lee\\nutcloud\\Data\\cfps\\2018\\cfps2018famconf_202008.dta',
 'C:\\Users\\lee\\nutcloud\\Data\\cfps\\2020\\cfps2020famconf_202301.dta']

In [219]:
result = []
for file_path in all_family_df_list[1:]:
    df = read_cfps(file_path)
    df = dr.rename(df,df.wav[0])
    result.append(process_mother_children(df))

df = pd.concat(result)

C:\Users\lee\nutcloud\Data\cfps\2014\cfps2014famconf_170630.dta
feather存在，读取feather。


C:\Users\lee\nutcloud\Data\cfps\2016\cfps2016famconf_201804.dta
feather存在，读取feather。
C:\Users\lee\nutcloud\Data\cfps\2018\cfps2018famconf_202008.dta
feather存在，读取feather。
C:\Users\lee\nutcloud\Data\cfps\2020\cfps2020famconf_202301.dta
feather存在，读取feather。


In [220]:
dr.select(df,2020)

['pid', 'wav']


,pid,wav
0,1.000515e+08,2014
3,1.101471e+08,2014
4,1.200091e+08,2014
8,1.301555e+08,2014
9,1.004534e+08,2014
...,...,...
51244,2.112291e+08,2020
51246,6.694542e+09,2020
51250,4.106501e+08,2020
51252,6.759192e+09,2020


## 正式开始


## 给定年份，制作母亲-子女对应表

In [506]:
dr = DataRenamer("./var_list.xlsx")

# 年份
year = os.environ.get('cfps_year', 2014)

# 家庭表
path = find_cfps(year,r'.*famconf')[0]
fam_df = read_cfps(path)
fam_df = dr.rename(fam_df,year)


C:\Users\lee\nutcloud\Data/cfps\2014\cfps2014famconf_170630.dta
feather存在，读取feather。
重命名信息:
{'pid': 'pid', 'wav': 'wav', 'wf3m': 'enroll', 'wc01': 'enroll2', 'wf502': 'math', 'cfps2014eduy_im': 'eduy', 'cfps2014_age': 'age', 'birthy': 'birth_y', 'qa701code': 'eth', 'cfps_gender': 'gender', 'tb2_a_p': 'gender', 'qa401ccode': 'birth_prov'}


In [507]:
# 母亲-子女表
mother_children_df = process_mother_children(fam_df)
mother_children_df.head()


,pid,wav,pid_c1,pid_c2,pid_c3,pid_c4,pid_c5,pid_c6,pid_c7,pid_c8,pid_c9,pid_c10
0,100051501.0,2014,110043107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,110147102.0,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,120009102.0,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,130155521.0,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,100453401.0,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [508]:
# 宽转长：子女母亲表
child_mother_df  = mother_children_df.melt(
    id_vars=['pid', 'wav'], 
    value_name='child_pid').drop(
        'variable',axis=1
        ).rename(columns={'pid':'mother_pid'}).dropna().query('child_pid > 100 & mother_pid > 100')
        
child_mother_df

,mother_pid,wav,child_pid
0,100051501.0,2014,110043107.0
5,430111102.0,2014,430111103.0
6,430111103.0,2014,100453401.0
8,130492103.0,2014,130492521.0
11,101129501.0,2014,130896521.0
...,...,...,...
245854,441105102.0,2014,441105110.0
246598,441697102.0,2014,441697106.0
254590,621472102.0,2014,621472101.0
274256,441105102.0,2014,441105111.0


In [509]:
person_df = read_cfps(find_cfps(year,r'(adult|person)')[0])


C:\Users\lee\nutcloud\Data/cfps\2014\cfps2014adult_201906.dta
feather存在，读取feather。


In [510]:
[x for x in person_df.columns if 'birth' in x]

['birthy']

In [511]:
# 成年人信息表
person_df = read_cfps(find_cfps(year,r'(adult|person)')[0])
person_df = dr.rename(person_df,year)
person_sub_df = dr.select(person_df)
person_sub_df.columns = 'child_' + person_sub_df.columns
person_sub_df.head()


C:\Users\lee\nutcloud\Data/cfps\2014\cfps2014adult_201906.dta
feather存在，读取feather。
重命名信息:
{'pid': 'pid', 'wav': 'wav', 'wf3m': 'enroll', 'wc01': 'enroll2', 'wf502': 'math', 'cfps2014eduy_im': 'eduy', 'cfps2014_age': 'age', 'birthy': 'birth_y', 'qa701code': 'eth', 'cfps_gender': 'gender', 'tb2_a_p': 'gender', 'qa401ccode': 'birth_prov'}
选择的列:
['pid', 'birth_y', 'age', 'gender', 'eth', 'enroll2', 'eduy', 'birth_prov', 'wav']


,child_pid,child_birth_y,child_age,child_gender,child_eth,child_enroll2,child_eduy,child_birth_prov,child_wav
0,441876105.0,1987.0,27.0,0.0,-8.0,0.0,9.0,NaN,2014
1,140093104.0,1992.0,22.0,1.0,-8.0,0.0,12.0,NaN,2014
2,130510104.0,1987.0,27.0,1.0,1.0,0.0,12.0,NaN,2014
3,130724103.0,1994.0,20.0,0.0,-8.0,1.0,12.0,-8.0,2014
4,110051102.0,1932.0,82.0,0.0,-8.0,-8.0,0.0,NaN,2014


In [512]:
# 幼儿信息表
child_proxy_df = read_cfps(find_cfps(year,'child')[0])
child_young_df = dr.rename_and_select(child_proxy_df,year)
child_young_df.head()

C:\Users\lee\nutcloud\Data/cfps\2014\cfps2014child_201906.dta
feather存在，读取feather。
重命名信息:
{'pid': 'pid', 'wav': 'wav', 'wf3m': 'enroll', 'wc01': 'enroll2', 'wf502': 'math', 'cfps2014eduy_im': 'eduy', 'cfps2014_age': 'age', 'birthy': 'birth_y', 'qa701code': 'eth', 'cfps_gender': 'gender', 'tb2_a_p': 'gender', 'qa401ccode': 'birth_prov'}
选择的列:
['pid', 'birth_y', 'age', 'gender', 'enroll', 'math', 'enroll2', 'eduy', 'wav']


,pid,birth_y,age,gender,enroll,math,enroll2,eduy,wav
0,210956105.0,2005.0,9.0,1.0,1.0,3.0,NaN,4.0,2014
1,441674401.0,2011.0,3.0,0.0,1.0,-8.0,NaN,0.0,2014
2,130385106.0,2008.0,6.0,0.0,1.0,1.0,NaN,0.0,2014
3,210955521.0,2012.0,2.0,1.0,5.0,-8.0,NaN,0.0,2014
4,113589503.0,2013.0,1.0,1.0,5.0,-8.0,NaN,0.0,2014


In [513]:


child_young_df.columns = 'child_' + child_young_df.columns
child_young_df.head()

,child_pid,child_birth_y,child_age,child_gender,child_enroll,child_math,child_enroll2,child_eduy,child_wav
0,210956105.0,2005.0,9.0,1.0,1.0,3.0,NaN,4.0,2014
1,441674401.0,2011.0,3.0,0.0,1.0,-8.0,NaN,0.0,2014
2,130385106.0,2008.0,6.0,0.0,1.0,1.0,NaN,0.0,2014
3,210955521.0,2012.0,2.0,1.0,5.0,-8.0,NaN,0.0,2014
4,113589503.0,2013.0,1.0,1.0,5.0,-8.0,NaN,0.0,2014


In [514]:
# 把成人和幼儿信息汇总：可能的子女信息表
# child_df = pd.concat([child_young_df,person_sub_df])
# child_df.head()

In [515]:
# child_df[~child_df['child_pid'].duplicated()].iloc[0,0]

In [516]:
# child_df.query('child_pid == 110026103.0')

In [526]:
from src.utils import safe_merge

In [518]:
x = safe_merge(child_mother_df,child_young_df)
x.head()

Merge的方式是how='left'
注意：合并后的 DataFrame 有 709/25061 个重复键值。第一个重复键值为: child_pid    110051104.0
Name: 156, dtype: float64


,mother_pid,wav,child_pid,child_birth_y,child_age,child_gender,child_enroll,child_math,child_enroll2,child_eduy,child_wav
0,100051501.0,2014,110043107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,430111102.0,2014,430111103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,430111103.0,2014,100453401.0,2012.0,2.0,0.0,1.0,-8.0,-8.0,0.0,2014.0
3,130492103.0,2014,130492521.0,2014.0,0.0,0.0,5.0,-8.0,-8.0,0.0,2014.0
4,101129501.0,2014,130896521.0,2013.0,1.0,1.0,5.0,-8.0,-8.0,0.0,2014.0


In [520]:
mother_df = dr.rename_and_select(person_df,year)
mother_df.columns = 'mother_' + mother_df.columns
mother_df.head()

重命名信息:
{'pid': 'pid', 'wav': 'wav', 'wf3m': 'enroll', 'wc01': 'enroll2', 'wf502': 'math', 'cfps2014eduy_im': 'eduy', 'cfps2014_age': 'age', 'birthy': 'birth_y', 'qa701code': 'eth', 'cfps_gender': 'gender', 'tb2_a_p': 'gender', 'qa401ccode': 'birth_prov'}
选择的列:
['pid', 'birth_y', 'age', 'gender', 'eth', 'enroll2', 'eduy', 'birth_prov', 'wav']


,mother_pid,mother_birth_y,mother_age,mother_gender,mother_eth,mother_enroll2,mother_eduy,mother_birth_prov,mother_wav
0,441876105.0,1987.0,27.0,0.0,-8.0,0.0,9.0,NaN,2014
1,140093104.0,1992.0,22.0,1.0,-8.0,0.0,12.0,NaN,2014
2,130510104.0,1987.0,27.0,1.0,1.0,0.0,12.0,NaN,2014
3,130724103.0,1994.0,20.0,0.0,-8.0,1.0,12.0,-8.0,2014
4,110051102.0,1932.0,82.0,0.0,-8.0,-8.0,0.0,NaN,2014


In [521]:
# 注意，1个母亲可以有多个孩子，因此母亲pid重复是没问题的
reg_df = child_mother_df.pipe(
    safe_merge, child_young_df).pipe(
        safe_merge, mother_df).query(
                    'child_age >=10 & child_age <=19').applymap(
                        lambda x: np.nan if (x < 0 or x == 79 or x == 77) else x)

len(reg_df)

Merge的方式是how='left'
注意：合并后的 DataFrame 有 709/25061 个重复键值。第一个重复键值为: child_pid    110051104.0
Name: 156, dtype: float64
Merge的方式是how='left'
注意：合并后的 DataFrame 有 8872/25061 个重复键值。第一个重复键值为: mother_pid    110051101.0
Name: 156, dtype: float64


2846

In [523]:
reg_df.head()

,mother_pid,wav,child_pid,child_birth_y,child_age,child_gender,child_enroll,child_math,child_enroll2,child_eduy,child_wav,mother_birth_y,mother_age,mother_gender,mother_eth,mother_enroll2,mother_eduy,mother_birth_prov,mother_wav
22,360160106.0,2014,360160109.0,1999.0,15.0,0.0,1.0,2.0,NaN,10.0,2014.0,1979.0,35.0,0.0,1.0,0.0,9.0,NaN,2014.0
24,370115104.0,2014,370115107.0,2004.0,10.0,1.0,1.0,1.0,1.0,3.0,2014.0,1981.0,33.0,0.0,NaN,0.0,9.0,NaN,2014.0
38,110011101.0,2014,110011103.0,1999.0,15.0,0.0,1.0,4.0,1.0,8.0,2014.0,1973.0,41.0,0.0,NaN,0.0,6.0,NaN,2014.0
44,110020105.0,2014,110020107.0,2001.0,13.0,0.0,1.0,3.0,1.0,7.0,2014.0,1970.0,44.0,0.0,NaN,0.0,15.0,NaN,2014.0
45,110022101.0,2014,110022103.0,2001.0,13.0,0.0,1.0,2.0,1.0,5.0,2014.0,1978.0,36.0,0.0,NaN,0.0,15.0,NaN,2014.0


### 统计性描述

In [524]:
from src.utils import desc
desc(reg_df.drop(['mother_pid','child_pid'],axis=1)).round(2)

,count,min,max,mean,mode
wav,2846,2014.0,2014.0,2014.00,2014.0
child_birth_y,2846,1999.0,2009.0,2001.50,2004.0
child_age,2846,10.0,15.0,12.51,10.0
child_gender,2846,0.0,1.0,0.54,1.0
child_enroll,2840,1.0,5.0,1.16,1.0
child_math,2711,1.0,4.0,2.27,1.0
child_enroll2,2486,0.0,1.0,0.98,1.0
child_eduy,2798,0.0,13.0,6.12,7.0
child_wav,2846,2014.0,2014.0,2014.00,2014.0
mother_birth_y,2631,1938.0,1988.0,1974.86,1976.0


In [525]:
reg_df.to_csv(f'data/reg_df_{year}.csv',index=False)